# Helper Notebook for Calling Functions

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')
import wines as w
wines = w.wines
from sklearn import linear_model

In [16]:
def make_heatmap(df):
    #create figure and axis
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    
    #create mask
    mask = np.zeros_like(df.corr(), dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    #create color scheme
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    return sns.heatmap(df.corr(),
                       mask=mask,
                       cmap=cmap,
                       center=0,
                       vmin=-0.4,
                       vmax=0.4,
                       square=True,
                       linewidths=0.5,
                       cbar_kws={'shrink': 0.5}, ax=ax);

In [17]:
def make_df(df, columns):
    _df = pd.DataFrame(df['points'])
    _df['scaled_log_prices'] = df['scaled_log']
    for col in columns:
        _df[col] = df[col]
    return pd.DataFrame(_df)

In [18]:
# def generate_wine_codes(df, province, grape, vintage, price):

        
#     return wine_codes

In [19]:
def generate_prediction(model, df, province, grape, vintage, price):

    # ------ Generate Unique Wine Codes ------ #   
    #log price
    log_price = int(round(np.log(price), 0))
    
    #create needed containers
    wine_codes = []
    
    # create a unique binary code for province
    for taster in df['taster_names']:    
        
        #create unique wine code for each taster 
        unique_wine_code = [] #resets code to empty list for each taster
        unique_wine_code.extend(df['provinces'][province])
        unique_wine_code.extend(df['taster_names'][taster])
        unique_wine_code.extend(df['grapes'][grape])
        unique_wine_code.extend(df['vintages'][vintage])
        unique_wine_code[-1] = log_price
          
        #create a list of unique wine codes
        wine_codes.append(unique_wine_code)
    # ---------------------------------------- #  
    
    
    # ------ Generate Prediction ------ #       
    #create needed containers
    predictions = []
    
    #loop through unique wine codes to predict what each taster would rate the win
    for wine_code in wine_codes:
        products = []
        for i in range(len(wine_code)):
            products.append(wine_code[i]*model.coef_[i])
        prediction = model.intercept_ + sum(products)
        predictions.append(prediction)
    
        #take the average predicted rating
        predicted_rating = round(np.mean(predictions), 2)
    # ---------------------------------- # 
    
    # ------ Generate Visualization ------ #   
    x_data = list((range(0,15)))
    x_axis_labels = ['Alexander Peartree',
                'Anna Lee C. Iijima',
                'Anne Krebiehl',
                'Carrie Dykes',
                'Jim Gordon',
                'Joe Czerwinski',
                'Kerin O’Keefe',
                'Lauren Buzzeo',
                'Matt Kettmann',
                'Paul Gregutt',
                'Roger Voss',
                'Sean P. Sullivan',
                'Susan Kostrzewa',
                'Undisclosed' ,
                'Virginia Boone']

    # format inputs
    year = str(vintage)
    grape = grape.replace('_', ' ').title()
    province = province.title()
    price = f'${price}'
    
    
    #create graph
    fig = plt.figure(figsize =(10,8))
    ax = fig.add_subplot(111)
    
    ax.bar(x_data, predictions, color='gray', alpha=0.7);
    ax.axhline(predicted_rating, color='green', linewidth=2, linestyle='--', label=f'Average Predicted Rating: {predicted_rating}');
    ax.set_title(f'Predicted Wine Enthusiast Ratings by Taster for a {price}\n{grape} from {province} bottled in {year}', {'fontsize' : 16});
    ax.set_ylabel('Rating', {'fontsize' : 14});
    ax.set_xlabel('Taster', {'fontsize' : 14});
    ax.set_ylim(80, 100);
                                                                                                                           
    plt.xticks(x_data, x_axis_labels, rotation=90);
    plt.legend(loc='upper right', fontsize='large');
    # ------------------------------------ # 
    
    return None